In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scipy
import csv

from scipy.stats import pearsonr


## Calculer la corrélation
La corrélation permet de vérifier que les données corrèlent. 

Pearson's correlation coefficient: $$\rho = \frac{\sum\limits_{i=1}^{n}\left(a_i - a_\mu\right)\left(b_i - b_\mu\right)}{\sigma_a \sigma_b}$$

Spearman's correlation coefficient: $$\rho = \frac{\sum\limits_{i=1}^{n}\left(a_{i,rank} - a_{\mu,rank}\right)\left(b_{i,rank} - b_{\mu,rank}\right)}{\sigma_{a,rank} \sigma_{b,rank}}$$

Calculer la corrélation entre la longeur des commentaires et la colère.
Théorie = plus la longueur du commentaire et longue plus c'est colère

In [36]:
df = pd.read_csv("../data/clean/commentaires.csv")

longueurs = df["text"].apply(lambda x: len(x.split()))

moyenne_longueurs = longueurs.mean()

print("La moyenne de la longueur des commentaires dans le fichier CSV est de : ", moyenne_longueurs)

La moyenne de la longueur des commentaires dans le fichier CSV est de :  20.985714285714284


In [37]:
liste_longueurs = []

for index, row in df.iterrows():
    if len(row["text"]) > moyenne_longueurs and row["label"] == "3":
        liste_longueurs.append(len(row["text"]))

if not longueurs:
    print("Pas de données suffisantes pour calculer la corrélation")
else:
    corr = np.corrcoef(longueurs, [3]*len(longueurs))[0,1]
    print("Corrélation entre la longueur des commentaires et le la colère :", corr)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [38]:
from scipy.stats import pearsonr

colere = df["label"].tolist()
moyenne_longueurs = int(moyenne_longueurs)


# Filtrage manuel des valeurs manquantes (facultatif, uniquement si vous avez des NaN ou des infinis)
# moyenne_longueurs_clean = [x for x, y in zip(moyenne_longueurs, colere) if x is not None and y is not None]
# colere_clean = [y for x, y in zip(moyenne_longueurs, colere) if x is not None and y is not None]

correlation, p_value = pearsonr(moyenne_longueurs, colere)
print("La corrélation de Pearson entre moyenne_longueurs et colere est de :", correlation)
print("La valeur-p de la corrélation est de :", p_value)


TypeError: object of type 'int' has no len()

## Calculer la p value
La p-value est une probabilité qui permet de calculer la significativité statistique.

## Le train test split
azqesrtdyuiojpk^lkjhgxfdsq